# NeuraLogic Hooks

In [1]:
import numpy as np

from neuralogic.core import Template, Backend, Atom, Var
from neuralogic.core.constructs.predicate import Predicate
from neuralogic.utils.data import Data, Dataset
from neuralogic.core.settings import Settings, ErrorFunction, Optimizer
from neuralogic.nn import get_evaluator

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Data preparation

In [2]:
src = np.array([
    1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10, 10, 11,
    12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21, 25, 25, 27, 27,
    27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32,
    32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33
])

dst = np.array([
    0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4, 5,
    0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23, 24, 2,
    23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23, 29, 30,
    31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30, 31, 32
])

u = np.concatenate([src, dst])
v = np.concatenate([dst, src])
indices = [[i] for i in range(34)]

train_dataset = Dataset(data=[Data(x=np.ones((34,)), edge_index=[u, v], y=[[-1], [1]], y_mask=[[0], [33]])])
test_dataset = Dataset(data=[Data(x=np.ones((34,)), edge_index=[u, v], y=indices, y_mask=indices)])

In [3]:
settings = Settings(epochs=2, error_function=ErrorFunction.SQUARED_DIFF, optimizer=Optimizer.SGD)

## Model preparation

In [4]:
template = Template(settings=settings)

with template.context():
    template.add_rules([(Atom.node_feature_embed(i)[10, 1] <= Atom.node_feature(i)) for i in range(32)])

    template.add_rule(Atom.gcn_1(Var.X)[10, 10] <= (Atom.node_feature_embed(Var.Y), Atom.edge(Var.X, Var.Y)))
    template.add_rule(Atom.gcn_2(Var.X)[10, 10] <= (Atom.gcn_1(Var.Y), Atom.edge(Var.X, Var.Y)))
    template.add_rule(Atom.predict(Var.X)[np.ones((1, 10))] <= (Atom.gcn_2(Var.X)))

print(template)

{10, 1} node_feature_embed(0) :- node_feature(0).
{10, 1} node_feature_embed(1) :- node_feature(1).
{10, 1} node_feature_embed(2) :- node_feature(2).
{10, 1} node_feature_embed(3) :- node_feature(3).
{10, 1} node_feature_embed(4) :- node_feature(4).
{10, 1} node_feature_embed(5) :- node_feature(5).
{10, 1} node_feature_embed(6) :- node_feature(6).
{10, 1} node_feature_embed(7) :- node_feature(7).
{10, 1} node_feature_embed(8) :- node_feature(8).
{10, 1} node_feature_embed(9) :- node_feature(9).
{10, 1} node_feature_embed(10) :- node_feature(10).
{10, 1} node_feature_embed(11) :- node_feature(11).
{10, 1} node_feature_embed(12) :- node_feature(12).
{10, 1} node_feature_embed(13) :- node_feature(13).
{10, 1} node_feature_embed(14) :- node_feature(14).
{10, 1} node_feature_embed(15) :- node_feature(15).
{10, 1} node_feature_embed(16) :- node_feature(16).
{10, 1} node_feature_embed(17) :- node_feature(17).
{10, 1} node_feature_embed(18) :- node_feature(18).
{10, 1} node_feature_embed(19) :

In [5]:
evaluator = get_evaluator(Backend.DYNET, template, settings)

There are no hooks set up - training/testing will work with no side effects

In [6]:
for _ in evaluator.train(train_dataset):
    pass

## Adding hooks

### Hooks declaration

Hooks are normal python functions with two parameters - `name` and `value`. `name` is the triggering name - which neuron triggered the hook, and the `value` corresponds to the output of the neuron that triggered the hook.

In [7]:
def my_gcn2_term_zero_hook(value):
    print("First hook: gcn_2(0) has value", value)


def my_gcn2_another_term_zero_hook(value):
    print("Second hook: gcn_2(0) has value", value)

    
def my_gcn1_hook(value):
    print("gcn_1(12) value: ", value)

### Attaching hooks

Hooks can be attached and detached to an atom in a template in multiple ways. All following ways are valid and will have the same result.

In [8]:
template.add_hook("gcn_2(0)", my_gcn2_term_zero_hook)

In [9]:
with template.context():
    template.add_hook(Atom.gcn_2(0), my_gcn2_term_zero_hook)

Detaching hooks on a predicate can be done similarly to attaching. #todo gusta: nezapomenout ze hooks jsou na Atom Values a ne na predikatech..

In [10]:
template.remove_hook("gcn_2(0)", my_gcn2_term_zero_hook)

In [11]:
with template.context():
    template.remove_hook(Atom.gcn_2(0), my_gcn2_term_zero_hook)

### Example

Attach our hooks:

In [12]:
with template.context():
    template.add_hook(Atom.gcn_2(0), my_gcn2_term_zero_hook)
    template.add_hook(Atom.gcn_2(0), my_gcn2_another_term_zero_hook)  # We can add multiple hooks to one predicate
    template.add_hook(Atom.gcn_1(12), my_gcn1_hook)

Forward propagation will now trigger hooks when the value for the hooked atom is being calculated

In [13]:
for _ in evaluator.train(train_dataset):
    print("\nEpoch trained\n")

gcn_1(12) value:  [[ 0.63730776]
 [ 0.12132919]
 [-0.6449492 ]
 [-0.97608614]
 [ 0.6988067 ]
 [ 0.61335289]
 [-0.55023974]
 [-0.19521706]
 [-0.88610041]
 [ 0.03012855]]
Second hook: gcn_2(0) has value [[ 0.14916514]
 [ 0.68800992]
 [ 0.66025007]
 [-0.67081004]
 [ 0.88787848]
 [-0.80579597]
 [-0.91197145]
 [ 0.02759976]
 [-0.3043721 ]
 [-0.15584695]]
First hook: gcn_2(0) has value [[ 0.14916514]
 [ 0.68800992]
 [ 0.66025007]
 [-0.67081004]
 [ 0.88787848]
 [-0.80579597]
 [-0.91197145]
 [ 0.02759976]
 [-0.3043721 ]
 [-0.15584695]]

Epoch trained

gcn_1(12) value:  [[ 0.6422106 ]
 [ 0.14667338]
 [-0.61853504]
 [-0.9760772 ]
 [ 0.69512385]
 [ 0.60692859]
 [-0.55196232]
 [-0.175337  ]
 [-0.88387311]
 [ 0.04793741]]
Second hook: gcn_2(0) has value [[ 0.22841918]
 [ 0.67398626]
 [ 0.62589133]
 [-0.68208307]
 [ 0.89479989]
 [-0.81070411]
 [-0.91395879]
 [-0.00815296]
 [-0.38714236]
 [-0.12913533]]
First hook: gcn_2(0) has value [[ 0.22841918]
 [ 0.67398626]
 [ 0.62589133]
 [-0.68208307]
 [ 0.89